In [393]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [394]:
# hack the paths, no package install needed
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [395]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')
import os

import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import pandas as pd

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

In [396]:
# # temp - check flux header
# from astropy.io import fits
# _, header = fits.getdata(
#     '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE102159189-421F42_20240408T221858.903110Z_00.00.fits',
#     header=True
# )
# header

### Authenticate

In [397]:
# type in your COSMOS credentials
from astroquery.esa.euclid.core import Euclid
Euclid.login()

INFO:astroquery:Login to Euclid TAP server


INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


User:  mwalms01
Password:  ········


INFO:astroquery:OK
INFO:astroquery:Login to Euclid data service
INFO:astroquery:OK
INFO:astroquery:Login to Euclid cutout service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid data service [astroquery.esa.euclid.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid cutout service [astroquery.esa.euclid.core]


INFO:astroquery:OK


INFO: OK [astroquery.utils.tap.core]


### Set Pipeline Options and Load External Targets

In [398]:
# or you can use this config to run for real

cfg_dict = dict(
    base_dir='/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline',
    name='latest',
    release_name=None,  # allow tiles from any release
    bands=['VIS', 'NIR_Y', 'NIR_J', 'NIR_H'],  
    auxillary_products=['MERPSF','MERRMS'],  # ,'MERBKG'
    release_priority=['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612' , 'F-006'],
    overwrite_fits=False,
    
    # ignore these
    download_dir=None,
    tile_dir=None,
    cutout_dir=None,
    fits_dir=None,
    sanity_dir=None
    
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc
pipeline.create_folders(cfg)  # fill in those missing paths


INFO:root:Saving to /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest


{'base_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline', 'name': 'latest', 'release_name': None, 'bands': ['VIS', 'NIR_Y', 'NIR_J', 'NIR_H'], 'auxillary_products': ['MERPSF', 'MERRMS'], 'release_priority': ['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612', 'F-006'], 'overwrite_fits': False, 'download_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest', 'tile_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles', 'cutout_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/cutouts', 'fits_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/cutouts/fits', 'sanity_dir': '/media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/sanity'}

In [399]:

# TODO use your own csv, it just needs the columns specified here
# external_targets = pd.read_csv('/media/home/my_workspace/combined_karina_strong_lens_classifications.csv', usecols=['id_str', 'target_ra', 'target_dec'])
# external_targets['target_field_of_view'] = 20  # arcseconds

lrg = pd.read_csv('/media/home/my_workspace/repos/bulk-euclid-cutouts/bulk_euclid/external_targets/LRGs_4sims_all.csv')
lrg = lrg.rename(columns={'ra': 'target_ra', 'dec': 'target_dec', 'ID': 'id_str'})
del lrg['Unnamed: 0']

lrg['category'] = 'lrg_master_list'

# also add the false positives
false_positives_desi = pd.read_csv('/media/home/my_workspace/repos/bulk-euclid-cutouts/bulk_euclid/external_targets/False_Positives_DESI.csv')
false_positives_desi['category'] = 'desi_false_positive'

# also add the known candidates
known = pd.read_csv('/media/home/my_workspace/repos/bulk-euclid-cutouts/bulk_euclid/external_targets/strong_lens_for_pipeline.csv')
known = known[known['final_classification'].isin(['A', 'B'])]  # drop the Cs, not plausible
known['category'] = 'known_lens_candidate'

# external_targets = lrg
# external_targets = pd.concat([known, lrg], axis=0).reset_index(drop=True)
external_targets = pd.concat([known, lrg, false_positives_desi], axis=0).reset_index(drop=True)



external_targets['target_field_of_view'] = 20  # arcseconds

In [400]:
external_targets['target_field_of_view']

0        20
1        20
2        20
3        20
4        20
         ..
11150    20
11151    20
11152    20
11153    20
11154    20
Name: target_field_of_view, Length: 11155, dtype: int64

In [401]:
len(external_targets)

11155

In [402]:
external_targets.nunique()

id_str                  10991
field                      13
target_ra               10602
target_dec              10604
final_classification        2
category                    3
vdisp                    5407
z                        5410
Unnamed: 0               5603
target_field_of_view        1
dtype: int64

In [403]:
# TODO Karina, why are these duplicated?
external_targets = external_targets.drop_duplicates(subset=['id_str'], keep='first')

In [404]:
len(external_targets)

10991

In [405]:
external_targets['category'].value_counts()

category
desi_false_positive     5603
lrg_master_list         5246
known_lens_candidate     142
Name: count, dtype: int64

In [406]:
assert not any(external_targets['id_str'].duplicated())

In [407]:
external_targets

,id_str,field,target_ra,target_dec,final_classification,category,vdisp,z,Unnamed: 0,target_field_of_view
0,EUCLJ215447.40+174003.7,Abell2390,328.697535,17.667696,A,known_lens_candidate,NaN,NaN,NaN,20
1,EUCLJ215405.95+181205.2,Abell2390,328.524807,18.201453,A,known_lens_candidate,NaN,NaN,NaN,20
2,EUCLJ002251.41-494843.9,Abell2764,5.714233,-49.812211,A,known_lens_candidate,NaN,NaN,NaN,20
3,EUCLJ002539.83-491515.5,Abell2764,6.415961,-49.254323,A,known_lens_candidate,NaN,NaN,NaN,20
4,EUCLJ002330.90-485506.4,Abell2764,5.878780,-48.918448,A,known_lens_candidate,NaN,NaN,NaN,20
...,...,...,...,...,...,...,...,...,...,...
11150,EUC_6097_DESI-sel_other,NaN,150.537346,2.669771,NaN,desi_false_positive,NaN,NaN,5598.0,20
11151,EUC_6103_DESI-sel_other,NaN,150.545603,2.668681,NaN,desi_false_positive,NaN,NaN,5599.0,20
11152,EUC_6108_DESI-sel_other,NaN,150.564119,2.640197,NaN,desi_false_positive,NaN,NaN,5600.0,20
11153,EUC_6122_DESI-sel_other,NaN,150.608120,2.704482,NaN,desi_false_positive,NaN,NaN,5601.0,20


### For each target, find a tile that covers it

In [408]:
targets_with_tiles = pipeline.get_matching_tiles(cfg, external_targets)
targets_with_tiles

INFO:astroquery:Query finished.
INFO:root:Found 2514 query results
INFO:root:Begin target/tile cross-match


INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Matched 10991 targets to 193 tiles
INFO:root:Targets with tile matches: 10834
/media/user/repos/bulk-euclid-cutouts/bulk_euclid/external_targets/pipeline.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets_with_tiles["tile_index"] = targets_with_tiles["tile_index"].astype(int)


,id_str,field,target_ra,target_dec,final_classification,category,vdisp,z,Unnamed: 0,target_field_of_view,tile_index,release_name,tile_ra,tile_ra_min,tile_ra_max,tile_dec_min,tile_dec_max,tile_dec
0,EUCLJ095929.92+021352.1,COSMOS,149.874700,2.231164,A,known_lens_candidate,NaN,NaN,NaN,20,101544256,CALBLOCK_PV-005_R2,149.841279,149.699490,149.983041,2.108355,2.391659,2.25
1,EUCLJ095951.04+021236.8,COSMOS,149.962680,2.210235,A,known_lens_candidate,NaN,NaN,NaN,20,101544257,CALBLOCK_PV-005_R2,150.091432,149.949642,150.233193,2.108355,2.391659,2.25
2,EUCLJ100056.78+021225.8,COSMOS,150.236610,2.207190,A,known_lens_candidate,NaN,NaN,NaN,20,101544258,CALBLOCK_PV-005_R2,150.341584,150.199795,150.483346,2.108355,2.391659,2.25
3,EUCLJ100013.92+022249.5,COSMOS,150.058040,2.380438,A,known_lens_candidate,NaN,NaN,NaN,20,101545697,CALBLOCK_PV-005_R2,150.115734,149.973917,150.257523,2.358354,2.641658,2.50
4,EUCLJ175032.01+633251.1,EDFN,267.633377,63.547552,A,known_lens_candidate,NaN,NaN,NaN,20,102157631,F-006,267.626746,267.023494,268.229934,63.232133,63.765406,63.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10829,EUC_6097_DESI-sel_other,NaN,150.537346,2.669771,NaN,desi_false_positive,NaN,NaN,5598.0,20,101547138,CALBLOCK_PV-005_R2,150.643379,150.501533,150.785198,2.608353,2.891658,2.75
10830,EUC_6103_DESI-sel_other,NaN,150.545603,2.668681,NaN,desi_false_positive,NaN,NaN,5599.0,20,101547138,CALBLOCK_PV-005_R2,150.643379,150.501533,150.785198,2.608353,2.891658,2.75
10831,EUC_6108_DESI-sel_other,NaN,150.564119,2.640197,NaN,desi_false_positive,NaN,NaN,5600.0,20,101547138,CALBLOCK_PV-005_R2,150.643379,150.501533,150.785198,2.608353,2.891658,2.75
10832,EUC_6122_DESI-sel_other,NaN,150.608120,2.704482,NaN,desi_false_positive,NaN,NaN,5601.0,20,101547138,CALBLOCK_PV-005_R2,150.643379,150.501533,150.785198,2.608353,2.891658,2.75


In [409]:
targets_with_tiles['release_name'].value_counts()

release_name
F-003_240612          7684
F-006                 2131
CALBLOCK_PV-005_R2     812
F-003_240321           207
Name: count, dtype: int64

In [410]:
targets_with_tiles['category'].value_counts()

category
desi_false_positive     5602
lrg_master_list         5151
known_lens_candidate      81
Name: count, dtype: int64

In [411]:
# targets_with_tiles = targets_with_tiles.sort_values('tile_index')[1000:] # [:1000]
# targets_with_tiles = targets_with_tiles[targets_with_tiles['tile_index'] == 101544256]  # standard tile
# targets_with_tiles = targets_with_tiles[targets_with_tiles['tile_index'] == 102158586]  # broken tile




The speed of the pipeline depends on the number of tiles to download. It takes a few minutes per tile.

In [412]:
targets_with_tiles['tile_index'].nunique(), len(targets_with_tiles)

(192, 10834)

In [413]:
assert not any(targets_with_tiles[['target_ra', 'target_dec']].isna().any())

### For every tile, download the flux and auxillary (PSF, RMS, BKG) data and make cutouts around each target in that tile

In [ ]:
pipeline.make_cutouts(cfg, targets_with_tiles)

INFO:root:Tile 101544256, 0 of 192
INFO:astroquery:Query finished.
INFO:root:Found 4 query results
INFO:root:Downloading all data for tile 101544256


INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Downloaded all data for tile 101544256
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band VIS
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_Y
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_J
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_H
INFO:root:Cutout data sliced for all bands, begin saving to disk
INFO:root:Saved cutouts for all targets in tile 101544256
INFO:root:Tile 101544257, 1 of 192
INFO:astroquery:Query finished.
INFO:root:Found 4 query results
INFO:root:Downloading all data for tile 101544257


INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Downloaded all data for tile 101544257
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band VIS
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_Y
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_J
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_H
INFO:root:Cutout data sliced for all bands, begin saving to disk
INFO:root:Saved cutouts for all targets in tile 101544257
INFO:root:Tile 101544258, 2 of 192
INFO:astroquery:Query finished.
INFO:root:Found 4 query results
INFO:root:Downloading all data for tile 101544258


...

INFO:root:EUC_MER_MOSAIC-NIR-H-RMS_TILE102159777-565A73_20240408T220903.926320Z_00.00.fits saved at /media/home/team_workspaces/Euclid-Consortium/data/strong_lensing/external_targets_pipeline/latest/tiles/EUC_MER_MOSAIC-NIR-H-RMS_TILE102159777-565A73_20240408T220903.926320Z_00.00.fits


Done.


INFO:root:Downloaded all data for tile 102159777
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band VIS
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_Y
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_J
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_H
INFO:root:Cutout data sliced for all bands, begin saving to disk
INFO:root:Saved cutouts for all targets in tile 102159777
INFO:root:Tile 102159195, 52 of 192
INFO:astroquery:Query finished.
INFO:root:Found 12 query results
INFO:root:Downloading all data for tile 102159195


INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Downloaded all data for tile 102159195
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band VIS
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_Y
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_J
INFO:root:Loaded tile, ready to slice
INFO:root:Cutout data sliced for band NIR_H
INFO:root:Cutout data sliced for all bands, begin saving to disk
CRITICAL:root:Error saving cutout for target EUC_18932_DESI-sel
CRITICAL:root:EUC_18932_DESI-sel.fits: RMS in VIS data is empty, likely a SAS error
CRITICAL:root:Error saving cutout for target EUC_18048_DESI-sel
CRITICAL:root:EUC_18048_DESI-sel.fits: RMS in VIS data is empty, likely a SAS error
CRITICAL:root:Error saving cutout for target EUC_18051_DESI-sel_spiral
CRITICAL:root:EUC_18051_DESI-sel_spiral.fits: RMS in VIS data is empty, likely a SAS error
